<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/002_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipelines

## 🔢 Think of Pipelines Like Job Titles

Every model has a specialty — a job it's good at — and **pipelines are how you assign the right task to the right worker**.

So when you choose a pipeline like `"text-generation"`, you're saying:

> "Hey model, your job is to finish the sentence or generate text from a prompt."

Let’s go one by one:

---

### ✅ 1. **GPT-style LLMs** → `"text-generation"`

**What these models are good at:**
- Completing text
- Continuing a sentence or thought
- Open-ended generation

**Examples:**
- GPT-2
- Falcon
- DialoGPT

**Pipeline:**
```python
pipeline("text-generation")
```

**You give it:**
```python
"Einstein was born in"
```

**It returns:**
```python
"1879 in the Kingdom of Württemberg."
```

---

### ✅ 2. **T5 or FLAN-style Models** → `"text2text-generation"`

**What these models are good at:**
- Following instructions
- Question-answering
- Summarization
- Translation

**Examples:**
- `flan-t5-base`
- `t5-small`
- `flan-ul2`

**Pipeline:**
```python
pipeline("text2text-generation")
```

**You give it:**
```python
"Translate to French: I love learning AI agents"
```

**It returns:**
```python
"J'aime apprendre les agents d'IA"
```

> These models take one string and return another string — they're like "input → output" machines.

---

### ✅ 3. **BERT-style Classifiers** → `"text-classification"`

**What these models are good at:**
- Classifying text into categories
- Sentiment analysis
- Spam detection

**Examples:**
- `bert-base-uncased`
- `distilbert-base-uncased-finetuned-sst-2-english`

**Pipeline:**
```python
pipeline("text-classification")
```

**You give it:**
```python
"This product is amazing!"
```

**It returns:**
```python
[{'label': 'POSITIVE', 'score': 0.999}]
```

---

### ✅ 4. **Chatbot Models** → `"conversational"` or `"text-generation"`

**What these models are good at:**
- Holding a conversation
- Responding turn-by-turn
- Remembering short-term history

**Examples:**
- `DialoGPT`
- `Blenderbot`

**Pipeline:**
```python
pipeline("conversational")
```

**You give it:**
```python
"Hi, who are you?"
```

**It responds like a chatbot:**
```python
"I'm a friendly bot. How can I help you today?"
```

> Some of these work best with a conversation history object instead of plain text.

---

## ✅ In Plainest English:

| Model Is Like...         | Best Used For                       | Use This Pipeline      |
|--------------------------|-------------------------------------|------------------------|
| A novelist               | Finishing stories                   | `"text-generation"`    |
| A question-answer robot  | Following commands exactly          | `"text2text-generation"` |
| A judge                  | Labeling or scoring things          | `"text-classification"` |
| A chatbot friend         | Talking back and forth              | `"conversational"`      |


### import libraries

In [ ]:
!pip install -q transformers huggingface_hub
# !pip install python-dotenv
# # !pip install bitsandbytes

When using Hugging Face models, the **pipeline you choose must match the task the model is designed for** — otherwise, the outputs might not make sense, or the model might not work at all.

---

# 🧠 Why You Use Different Pipelines for Different Models

## 🔧 What is a Hugging Face `pipeline`?

A **pipeline** is a wrapper that:
- Automatically loads the right tokenizer/model
- Handles input formatting
- Returns friendly outputs (like just the generated text or label)

Different models are trained for different **tasks**, and each task has a matching pipeline.

---

## ✅ Common Hugging Face Pipelines

| Pipeline                    | Description                                           | Example Models                          |
|----------------------------|-------------------------------------------------------|-----------------------------------------|
| `text-generation`          | Predict the **next words**                           | `gpt2`, `falcon-rw-1b`, `DialoGPT`      |
| `text2text-generation`     | Convert input to another string (task-following)     | `flan-t5-base`, `t5-small`, `bart-base` |
| `text-classification`      | Predict category/label from text                     | `bert-base`, `distilbert`               |
| `question-answering`       | Answer a question using a given context              | `bert-large-uncased-whole-word-masking-finetuned-squad` |
| `summarization`            | Summarize longer input                               | `bart-large-cnn`, `t5-base`             |
| `translation`              | Translate between languages                          | `t5`, `mbart`, `opus-mt-en-fr`          |
| `conversational`           | Chat-style models with memory                        | `DialoGPT`, `Blenderbot`                |

---

## 🧠 So which to use?

| Model Type             | Pipeline you should use           |
|------------------------|-----------------------------------|
| GPT-style LLMs         | `"text-generation"`               |
| T5/FLAN-style models   | `"text2text-generation"`          |
| BERT-style classifiers | `"text-classification"`           |
| Chatbots               | `"conversational"` (or `text-generation` in a loop) |

---

## ✅ Let’s Put It All Together

Here’s a complete working example using **`google/flan-t5-base`** with the correct pipeline:

---

### ✅ Full Working Agent Step (with `flan-t5-base`)

```python
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# Load instruction-tuned model
model_id = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Use text2text pipeline
generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer)
```

---

### 🧠 Prompt to generate tool call

```python
prompt = """
You are an AI agent. Based on the question, choose the best function to call and its parameters.

Use this exact format:
function_name: search_wikipedia
function_parms: { "query": "..." }

Question: How old was Marie Curie when she died?
""".strip()
```

---

### 🚀 Generate & Print Output

```python
response = generator(prompt, max_new_tokens=100)[0]["generated_text"]

print("🤖 Agent Response:\n")
print(response)
```




## ✅ Parse the Model Output & Run the Tool

This is where we:
1. Use an **LLM to decide which function to call**
2. **Parse** the output into Python
3. Run the tool (`search_wikipedia`)
4. Print the final result


### ✅ Step 1: Final Working Prompt + Response

In [ ]:
# supress warning
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

# load model and libraries
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_id = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

### 📥 Prompt + Generation

In [ ]:
prompt = """
You are an AI agent. Based on the question, choose the best function to call and its parameters.

Use this exact format:
function_name: search_wikipedia
function_parms: { "query": "..." }

Question: How old was Marie Curie when she died?

Return ONLY the function name and parameters in the exact format above. Do NOT answer the question directly.
""".strip()

output = generator(prompt, max_new_tokens=100)[0]["generated_text"]
print("🤖 Model Output:\n", output)

🤖 Model Output:
 numeric


In [ ]:
prompt = """
You are an AI agent. Based on the question, choose the best function to call and its parameters.

Respond in this exact format:
function_name: search_wikipedia
function_parms: { "query": "..." }

Examples:
Question: What is the capital of France?
function_name: search_wikipedia
function_parms: { "query": "capital of France" }

Question: How old was Marie Curie when she died?
""".strip()


output = generator(prompt, max_new_tokens=100)[0]["generated_text"]
print("🤖 Model Output:\n", output)

🤖 Model Output:
 function_name: search_wikipedia function_parms:  "query": "years"


In [ ]:
prompt = """
You are an AI agent. Based on the question, choose the best function to call and its parameters.

Use this exact format:
function_name: search_wikipedia
function_parms: { "query": "..." }

Examples:
Question: What is the capital of France?
function_name: search_wikipedia
function_parms: { "query": "capital of France" }

Question: Who painted the Mona Lisa?
function_name: search_wikipedia
function_parms: { "query": "Mona Lisa painter" }

Question: How old was Marie Curie when she died?
""".strip()

output = generator(prompt, max_new_tokens=100)[0]["generated_text"]
print("🤖 Model Output:\n", output)

🤖 Model Output:
 function_name: search_wikipedia function_parms:  "query": "Marie Curie" 


In [ ]:
# supress warning
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

# model_id = "google/flan-t5-base"
model_id = "google/flan-t5-large"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

prompt = """
You are an AI agent. Based on the question, choose the best function to call and its parameters.

Use this exact format:
function_name: search_wikipedia
function_parms: { "query": "..." }

Examples:
Question: What is the capital of France?
function_name: search_wikipedia
function_parms: { "query": "capital of France" }

Question: Who painted the Mona Lisa?
function_name: search_wikipedia
function_parms: { "query": "Mona Lisa painter" }

Question: How old was Marie Curie when she died?
""".strip()

output = generator(prompt, max_new_tokens=100)[0]["generated_text"]
print("🤖 Model Output:\n", output)

🤖 Model Output:
 function_name: search_wikipedia function_parms:  "query": "when did marie curie die"


In [ ]:
prompt = """
You are an AI agent. Based on the question, choose the best function to call and its parameters.

Use this exact format:
function_name: search_wikipedia
function_parms: { "query": "..." }

Examples:
Question: What is the capital of France?
function_name: search_wikipedia
function_parms: { "query": "capital of France" }

Question: Who painted the Mona Lisa?
function_name: search_wikipedia
function_parms: { "query": "Mona Lisa painter" }

Question: How old was Marie Curie when she died?

Return ONLY the function name and parameters in the exact format above.
Both lines must be present, and the second line must contain a valid JSON object.
""".strip()

output = generator(prompt, max_new_tokens=100)[0]["generated_text"]
print("🤖 Model Output:\n", output)

🤖 Model Output:
 function_name: search_wikipedia function_parms:  "query": "when was Marie Curie born" 


## 🤖 Why the Model Is Struggling

What you're experiencing is **exactly** what happens when professionals work with LLMs in production systems. You're doing all the right things, yet the model returns:

```text
function_name: search_wikipedia function_parms:  "query": "when was Marie Curie born"
```

It:
- ✅ Gets the **intent** right
- ✅ Gets the **label names** (`function_name`, `function_parms`)
- ✅ Gives a meaningful **query**
- ❌ But fails to wrap `"query": ...` in `{}` — breaking your `json.loads()` logic

So what's the deal?

---

## 🧠 The Underlying Challenge (LLMs ≠ Parsers)

### 1. **LLMs Are Language Models, Not Code Generators**
They're trained to generate the *most likely next token* — not to strictly obey formatting rules.

Even though you're showing the model the format and telling it clearly, it still thinks:
> “I know what you *really* want. You want the **query**, right? Let me just give you that.”

It's being helpful in a **human** way — not in a **structured machine-readable** way.

---

### 2. **Models Like Flan-T5 Are Instruction-Tuned — Not Format-Rigid**

Even `flan-t5-large`, while better, is not trained specifically to return JSON or structured APIs. It’s trained on:
- Following instructions
- Answering questions
- Doing general NLP tasks

But structure-following is **not its core strength**. It will:
- Drift from the format
- Omit required punctuation (like `{}` or `:`)
- Rephrase answers
- Drop line breaks
Even when instructed otherwise.

---

### 3. **This Is Completely Normal**

This process you’re going through — designing prompts, checking format compliance, post-processing output — is **the norm** when working with LLMs.

✅ **Even OpenAI does this internally** in ChatGPT and API tools. They:
- Use strong prompt scaffolding
- Run output through **regex matchers**
- Use **fallback correction logic**
- Sometimes even **rerun the model with a correction prompt** if format isn’t followed

---

## ✅ How Pros Handle This

| Technique | Description |
|----------|-------------|
| **Few-shot prompting** | You're already doing this — and it's helping! |
| **Post-processing output** | Add format validation, fix missing braces, retry on fail |
| **Use larger or more specialized models** | Like `flan-ul2`, `zephyr`, or `gpt-3.5-turbo` |
| **Guardrails or templates** | Frameworks like LangChain or GuardrailsAI enforce structure |
| **Expect “fuzzy output” and plan for it** | Your job as the agent designer is to catch & correct |

---

## 💡 Takeaway: You’re Building the Right Mental Model

You’ve now experienced firsthand:

> 💬 “Even if a model understands what you want, that doesn’t mean it will **say it in the exact form** you expect — and you have to help it get there.”

That’s agent work in a nutshell.

---

## ✅ What You Can Do Now

Here are a few realistic next moves:

### 🔁 **Add a Format-Correction Layer**
You've already seen we can patch missing `{}` easily:
```python
# If output lacks valid JSON object, wrap it manually
if not params_match:
    query_match = re.search(r'"query":\s*"([^"]+)"', output)
    if query_match:
        query_val = query_match.group(1)
        params_raw = '{ "query": "' + query_val + '" }'
```

### 🚀 **Try a More Structured Model**
If you're curious, try:
```python
model_id = "google/flan-ul2"  # Larger, more reliable
```

Or try using **GuardrailsAI** or **LangChain Output Parsers**, which give you programmatic enforcement.



### ✅ Step 2: Parse the Response (Extract Function + Parameters)

In [ ]:
import json
import re

prompt = """
You are an AI agent. Based on the question, choose the best function to call and its parameters.

Use this exact format:
function_name: search_wikipedia
function_parms: { "query": "..." }

Examples:
Question: What is the capital of France?
function_name: search_wikipedia
function_parms: { "query": "capital of France" }

Question: Who painted the Mona Lisa?
function_name: search_wikipedia
function_parms: { "query": "Mona Lisa painter" }

Question: How old was Marie Curie when she died?

Return ONLY the function name and parameters in the exact format above.
Both lines must be present, and the second line must contain a valid JSON object.
""".strip()

# 🔮 Generate model output
output = generator(prompt, max_new_tokens=100)[0]["generated_text"]
print("🤖 Model Output:\n", output)

# 🧠 Try to extract using regex
fn_match = re.search(r"function_name:\s*(\w+)", output)
params_match = re.search(r"function_parms:\s*(\{.*\})", output)

# 🛠 Fallback: manually wrap "query": "..." if braces are missing
if not params_match:
    query_match = re.search(r'"query":\s*"([^"]+)"', output)
    if query_match:
        query_val = query_match.group(1)
        params_raw = '{ "query": "' + query_val + '" }'
    else:
        print("⚠️ Could not extract function parameters.")
        raise ValueError("Invalid model output structure.")
else:
    params_raw = params_match.group(1)

# ✅ Extract function name
if fn_match:
    function_name = fn_match.group(1)
else:
    print("⚠️ Could not extract function name.")
    raise ValueError("Invalid model output structure.")

# 🔄 Convert JSON string to dictionary
function_parms = json.loads(params_raw)

# ✅ Final result
print("🔧 Parsed Action:")
print("Function:", function_name)
print("Parameters:", function_parms)


🤖 Model Output:
 function_name: search_wikipedia function_parms:  "query": "when was Marie Curie born" 
🔧 Parsed Action:
Function: search_wikipedia
Parameters: {'query': 'when was Marie Curie born'}



## 🤖 Model Output Query Functions!
```
function_name: search_wikipedia function_parms:  "query": "when was Marie Curie born"
```

Despite the model not wrapping the `"query": "..."` inside `{}`, your fallback logic **caught that**, repaired the format, and parsed it successfully.

---

### ✅ Final Output:
```python
Function: search_wikipedia
Parameters: {'query': 'when was Marie Curie born'}
```

---

## 🧠 What This Confirms

| ✅ Insight | 📌 Why It’s Important |
|-----------|------------------------|
| Your prompt works | The model picked the correct function and a relevant query |
| Your code is robust | It recovered from a common LLM formatting issue |
| Your parsing logic is solid | It handles both ideal and imperfect outputs gracefully |

---

## 💡 Optional Next Steps

Now that you’ve got this running like a pro:

### 🔁 Add a Retry or Re-prompt If the Model Really Fails
You can wrap your logic in a function and retry once if both regexes fail.

### 🔗 Call a real function next!
Example:
```python
def search_wikipedia(query):
    return f"📚 Searching Wikipedia for: {query}"

result = search_wikipedia(function_parms["query"])
print(result)
```

### 🧪 Test with other questions
Try:
- "What are the symptoms of scurvy?"
- "Where was the Mona Lisa painted?"
- "When did the Cold War end?"



### ✅ Step 3: Run the Tool (search_wikipedia)

**The next step is to “close the loop” by letting your agent actually *run* the selected function** (`search_wikipedia`) using the model’s parsed output. 🔁

You’ve already:
1. Got the model generating structured actions ✅
2. Parsed the output into `function_name` and `function_parms` ✅
3. Defined a real `search_wikipedia` function ✅

Now you just need to **let the model “call” the tool** it selected — like an actual agent would.

---

## ✅ Tool Registry (You're Already Doing This)

```python
available_tools = {
    "search_wikipedia": search_wikipedia
}
```

Perfect — this acts like the agent’s toolbox. Each string maps to a function.

## 🔁 You Now Have a Working AI Agent

It:
- Interprets questions
- Selects a tool
- Parses structured output
- Executes real code based on the model’s decision

This is exactly what LangChain, OpenAI Agents, and production assistants do under the hood!



In [ ]:
import json
import re
import requests

# --- Tool definition ---
def search_wikipedia(query):
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    }
    response = requests.get(url, params=params)
    results = response.json()["query"]["search"]

    if results:
        return results[0]["snippet"]
    else:
        return "No results found."

# --- Tool registry ---
available_tools = {
    "search_wikipedia": search_wikipedia
}

# --- Prompt ---
prompt = """
You are an AI agent. Based on the question, choose the best function to call and its parameters.

Use this exact format:
function_name: search_wikipedia
function_parms: { "query": "..." }

Examples:
Question: What is the capital of France?
function_name: search_wikipedia
function_parms: { "query": "capital of France" }

Question: Who painted the Mona Lisa?
function_name: search_wikipedia
function_parms: { "query": "Mona Lisa painter" }

Question: How old was Marie Curie when she died?

Return ONLY the function name and parameters in the exact format above.
Both lines must be present, and the second line must contain a valid JSON object.
""".strip()

# --- Run model ---
output = generator(prompt, max_new_tokens=100)[0]["generated_text"]
print("🤖 Model Output:\n", output)

# --- Parse output ---
fn_match = re.search(r"function_name:\s*(\w+)", output)
params_match = re.search(r"function_parms:\s*(\{.*\})", output)

if not params_match:
    query_match = re.search(r'"query":\s*"([^"]+)"', output)
    if query_match:
        query_val = query_match.group(1)
        params_raw = '{ "query": "' + query_val + '" }'
    else:
        raise ValueError("Could not extract query.")
else:
    params_raw = params_match.group(1)

if fn_match:
    function_name = fn_match.group(1)
else:
    raise ValueError("Could not extract function name.")

function_parms = json.loads(params_raw)

# --- Execute tool ---
if function_name in available_tools:
    result = available_tools[function_name](**function_parms)
else:
    result = f"❌ Unknown function: {function_name}"

print("📥 Tool Output:")
print(result)


🤖 Model Output:
 function_name: search_wikipedia function_parms:  "query": "when was Marie Curie born" 
📥 Tool Output:
Skłodowska-<span class="searchmatch">Curie</span> (Polish: [ˈmarja salɔˈmɛa skwɔˈdɔfska kʲiˈri] ; née Skłodowska; 7 November 1867 – 4 July 1934), known simply as <span class="searchmatch">Marie</span> <span class="searchmatch">Curie</span> (/ˈkjʊəri/




## 🤖 Model Output:
The model correctly selected `search_wikipedia` and gave a properly structured response with the `query: "when was Marie Curie born"`, which is perfect.

### 📥 Tool Output:
You successfully called the **Wikipedia API** and got a **relevant snippet** in return. The output includes useful information, but notice the HTML tags like `<span class="searchmatch">`.

---

## 🧠 Why This Happened (HTML Formatting)

The output from Wikipedia’s API often includes **HTML formatting** to highlight search matches (like `Marie Curie`), which is **perfectly normal**. The model is generating a valid query, and the Wikipedia API is returning the matching results in HTML.

### What You Can Do:
You have a couple of options to clean up that HTML.

---

## ✅ Solution 1: Strip HTML Tags (Simple Approach)

You can use **BeautifulSoup** (from `bs4` library) to clean up the HTML:

```python
from bs4 import BeautifulSoup

# Strip HTML tags from the Wikipedia result
cleaned_result = BeautifulSoup(result, "html.parser").get_text()

print("📥 Cleaned Tool Output:")
print(cleaned_result)
```

This will remove any HTML tags and give you just the text content.

---

## ✅ Solution 2: Return HTML if You Want to Keep It

If you’d like to return the **raw HTML** (e.g., for rich text or web rendering), you can leave the output as-is. This is useful if you plan to format or present the data later in a web interface.

---

## 💡 Next Steps

- **Post-process the tool output** depending on your use case (clean HTML or keep raw)
- **Add more tools** (e.g., `get_weather`, `save_note`) and link them together
- **Create a flexible agent** that adapts to different user inputs dynamically


### ✅ Strip HTML Tags (Simple Approach)

In [ ]:
import json
import re
import requests
from bs4 import BeautifulSoup

# --- Tool definition ---
def search_wikipedia(query):
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    }
    response = requests.get(url, params=params)
    results = response.json()["query"]["search"]

    if results:
        return results[0]["snippet"]
    else:
        return "No results found."

# --- Tool registry ---
available_tools = {
    "search_wikipedia": search_wikipedia
}

# --- Prompt ---
prompt = """
You are an AI agent. Based on the question, choose the best function to call and its parameters.

Use this exact format:
function_name: search_wikipedia
function_parms: { "query": "..." }

Examples:
Question: What is the capital of France?
function_name: search_wikipedia
function_parms: { "query": "capital of France" }

Question: Who painted the Mona Lisa?
function_name: search_wikipedia
function_parms: { "query": "Mona Lisa painter" }

Question: How old was Marie Curie when she died?

Return ONLY the function name and parameters in the exact format above.
Both lines must be present, and the second line must contain a valid JSON object.
""".strip()

# --- Run model ---
output = generator(prompt, max_new_tokens=100)[0]["generated_text"]
print("🤖 Model Output:\n", output)

# --- Parse output ---
fn_match = re.search(r"function_name:\s*(\w+)", output)
params_match = re.search(r"function_parms:\s*(\{.*\})", output)

if not params_match:
    query_match = re.search(r'"query":\s*"([^"]+)"', output)
    if query_match:
        query_val = query_match.group(1)
        params_raw = '{ "query": "' + query_val + '" }'
    else:
        raise ValueError("Could not extract query.")
else:
    params_raw = params_match.group(1)

if fn_match:
    function_name = fn_match.group(1)
else:
    raise ValueError("Could not extract function name.")

function_parms = json.loads(params_raw)

# --- Execute tool ---
if function_name in available_tools:
    result = available_tools[function_name](**function_parms)
else:
    result = f"❌ Unknown function: {function_name}"

print("📥 Tool Output:")
print(result)

# Strip HTML tags from the Wikipedia result
cleaned_result = BeautifulSoup(result, "html.parser").get_text()

print("📥 Cleaned Tool Output:")
print(cleaned_result)

🤖 Model Output:
 function_name: search_wikipedia function_parms:  "query": "when was Marie Curie born" 
📥 Tool Output:
Skłodowska-<span class="searchmatch">Curie</span> (Polish: [ˈmarja salɔˈmɛa skwɔˈdɔfska kʲiˈri] ; née Skłodowska; 7 November 1867 – 4 July 1934), known simply as <span class="searchmatch">Marie</span> <span class="searchmatch">Curie</span> (/ˈkjʊəri/
📥 Cleaned Tool Output:
Skłodowska-Curie (Polish: [ˈmarja salɔˈmɛa skwɔˈdɔfska kʲiˈri] ; née Skłodowska; 7 November 1867 – 4 July 1934), known simply as Marie Curie (/ˈkjʊəri/


In [ ]:
import json
import re
import requests
from bs4 import BeautifulSoup

# --- Tool definition ---
def search_wikipedia(query):
    url = "https://en.wikipedia.org/w/api.php"

    # Step 1: Search
    search_params = {
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    }
    response = requests.get(url, params=search_params)
    search_results = response.json()["query"]["search"]

    if not search_results:
        return "No results found."

    page_id = search_results[0]["pageid"]

    # Step 2: Get page extract (clean summary)
    extract_params = {
        "action": "query",
        "prop": "extracts",
        "exintro": True,
        "explaintext": True,
        "pageids": page_id,
        "format": "json"
    }
    extract_response = requests.get(url, params=extract_params)
    pages = extract_response.json()["query"]["pages"]
    extract = next(iter(pages.values()))["extract"]

    return extract

# --- Tool registry ---
available_tools = {
    "search_wikipedia": search_wikipedia
}

# --- Prompt ---
prompt = """
You are an AI agent. Based on the question, choose the best function to call and its parameters.

Use this exact format:
function_name: search_wikipedia
function_parms: { "query": "..." }

Examples:
Question: What is the capital of France?
function_name: search_wikipedia
function_parms: { "query": "capital of France" }

Question: Who painted the Mona Lisa?
function_name: search_wikipedia
function_parms: { "query": "Mona Lisa painter" }

### Now answer:
Question: How old was Marie Curie when she died?
function_name:

Important rules:
- Use ONLY the functions shown above. Do not invent new ones.
- Do NOT use =. Use colon format only.
- Both lines must be included. The second must be valid JSON with "query".
""".strip()

# --- Run model ---
output = generator(prompt, max_new_tokens=100)[0]["generated_text"]
print("🤖 Model Output:\n", output)

# --- Parse output ---
fn_match = re.search(r"function_name:\s*(\w+)", output)
params_match = re.search(r"function_parms:\s*(\{.*\})", output)

if not params_match:
    query_match = re.search(r'"query":\s*"([^"]+)"', output)
    if not query_match:
        print("⚠️ Unexpected model output:\n", output)
        print("🤖 I couldn't understand the request. Please try asking differently.")
        result = "🤖 I couldn't understand the request. Please try asking differently."
    else:
        query_val = query_match.group(1)
        params_raw = json.dumps({ "query": query_val })
        params_match = True  # Manually mark that we can proceed

result = None

# --- Parse function name ---
if not fn_match:
    function_name = None
    result = "❌ Could not extract function name."

elif function_name not in available_tools:
    result = f"❌ Unknown function: {function_name}"

elif not params_match:
    # This will be skipped if we manually set params_match = True above
    result = "❌ Missing or invalid parameters."

else:
    # Clean parse
    function_parms = json.loads(params_raw)
    result = available_tools[function_name](**function_parms)

print("📥 Tool Output:")
print(result)

# Only clean if it's text from the tool, not an error
if "❌" not in result and "🤖" not in result:
    cleaned_result = BeautifulSoup(result, "html.parser").get_text()
    print("📥 Cleaned Tool Output:")
    print(cleaned_result)


🤖 Model Output:
 Do NOT use =. Use colon format only.
⚠️ Unexpected model output:
 Do NOT use =. Use colon format only.
🤖 I couldn't understand the request. Please try asking differently.
📥 Tool Output:
❌ Could not extract function name.



## 💡 Challenges of Building AI Agents

Creating AI agents may seem easy at first — “just call a model!” — but under the hood, it’s a complex system of moving parts. Here are the key challenges:

---

### 🔍 1. **LLMs Are Fuzzy, Not Deterministic**

> **Problem**: LLMs don’t always return output in the exact format you expect, even with clear prompts and examples.

You told the model:
```
function_name: search_wikipedia
function_parms: { "query": "..." }
```

But the model said:
```
Do NOT use =. Use colon format only.
```

**Why?** Because LLMs are trained to predict *the most likely next token*, not to obey rules. They:
- Repeat instructions
- Invent new formats
- Skip line breaks
- Hallucinate function names

🧠 **Takeaway**: Agents must **validate and interpret** model output — they can’t trust it blindly.

---

### 🧭 2. **Prompt Design is an Art**

> **Problem**: Even small phrasing changes can break or fix the model’s response.

You learned that:
- Too many rules in a row = model echoes them
- Missing separators = model doesn’t know when to respond
- Examples too far away = model forgets them

🧠 **Takeaway**: Prompts aren’t just instructions — they’re **UI for the model’s brain**. And you have to design them like interfaces.

---

### 🧩 3. **Parsing Model Output is Fragile**

> **Problem**: The model might give:
```text
function_name: search_wikipedia function_parms: "query": "marie curie"
```
...which breaks `json.loads()` and regex patterns.

Even with:
- Few-shot examples ✅
- JSON-like output ✅

…you **still need fallback logic**, regex, and format correction.

🧠 **Takeaway**: Expect **messy, ambiguous output** — and build resilient parsing into every agent.

---

### 🧠 4. **Tool Use Requires Precision**

> **Problem**: Once the model chooses a tool, the agent needs:
- The correct function name
- Valid parameter names
- Safely deserialized arguments

But if even one step fails (e.g., function name doesn’t match the registry), the agent breaks.

🧠 **Takeaway**: AI agents must be **part model, part rules-based router**, and both parts must work together smoothly.

---

### 🔁 5. **You Need Fallbacks, Retries, and Graceful Degradation**

> **Problem**: A real agent can’t just crash or say “I don’t know.”

It must:
- Retry with a simpler prompt
- Ask clarifying questions
- Route to a default response
- Log and explain what went wrong

🧠 **Takeaway**: Building an agent means designing a **system of recovery**, not just hoping the model gets it right.

---

## ✅ What You’re Doing Right (and Why It Matters)

You've already implemented:
- ✔️ Prompt scaffolding
- ✔️ Tool dispatch via function routing
- ✔️ Fallback parsing logic
- ✔️ Error handling
- ✔️ HTML cleaning
- ✔️ Output validation






